In [1]:
import pandas as pd
import glob
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
matches_list1 = pd.read_excel('/Users/roumyadas/Desktop/IPL_Simulation/Fixtures/IPL_2024_schedule.xlsx',
                            sheet_name='Season_01')
matches_list2 = pd.read_excel('/Users/roumyadas/Desktop/IPL_Simulation/Fixtures/IPL_2024_schedule.xlsx',
                            sheet_name='Season_02')

matches_list1['match_id'] = matches_list1['num'].apply(lambda x: 'M00' + str(x) if x<=9 else 'M0' + str(x))
matches_list2['match_id'] = matches_list2['num'].apply(lambda x: 'S02M00' + str(x) if x<=9 else 'S02M0' + str(x))

matches_list = pd.concat([matches_list1[['num','Spin','match_id']],matches_list2[['num','Spin','match_id']]],
                         axis=0)



matches_list.columns
#matches_list = matches_list[['Date','Team One','Team Two']]

Index(['num', 'Spin', 'match_id'], dtype='object')

matches_list

In [3]:
#df_sim = pd.read_csv('/Users/roumyadas/Desktop/IPL_Simulation/Season_01/Stats//df_all_round_sim.csv')
df_sim2 = pd.read_csv('/Users/roumyadas/Desktop/IPL_Simulation/Season_02/df_all_round_sim.csv')

#df_sim = pd.concat([df_sim,df_sim2], axis=0).reset_index(drop=True)
df_sim = df_sim2

/Users/roumyadas/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (42) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
df_01 = pd.read_csv('/Users/roumyadas/Desktop/IPL_Simulation/Season_01/Stats/df_all_round_sim.csv')
df_curr = pd.DataFrame(columns=df_01.columns.tolist())
try:
    df_curr = pd.read_csv('/Users/roumyadas/Desktop/IPL_Simulation/Season_02/df_all_round_sim.csv')
    #df_curr = df_curr[df_01.columns.tolist()]
except FileNotFoundError:
    print("df_curr file not found, proceeding with df_01 only.")

df_all = pd.concat([df_01,df_curr], axis=0)
df_all.reset_index(drop=True, inplace=True)


df_all['phase'] = np.where(df_all['legal_balls_bowled']<=36, 'pp', 
                        np.where(df_all['legal_balls_bowled']>=90, 'death',
                            'middle'))

df_sim = df_all

([1, 2, 6, 1, 6, 1, 2, 4, 3, 4, 3, 4, 3, 4, 3, 5, 2, 5, 2, 5],
 [1, 5, 1, 6, 5, 1, 6, 3, 4, 6, 3, 6, 3, 4, 3, 2, 5, 2, 5, 2])

df_sim[df_sim['batting_team']=='CSK']['striker'].unique()

In [5]:
play_style_df = pd.read_csv("/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/play_style.csv")


In [6]:
df_sim.loc[df_sim.isWicket==1, 'partnership_runs'] = np.nan

df_sim['partnership_runs'] = df_sim['partnership_runs'].fillna(method='ffill')
df_sim.loc[df_sim.isWicket==1, 'partnership_runs'] = df_sim.loc[df_sim.isWicket==1, 'partnership_runs']+\
                                            df_sim.loc[df_sim.isWicket==1, 'total_runs']

In [7]:
#samp = df_sim[df_sim.match_id=='M072']

df_sim['before_ball_partnership'] = 0
df_sim['after_ball_partnership'] = 0
df_sim['before_balls'] = 0
df_sim['after_balls'] = 0

for m_id in df_sim.match_id.unique():
    #print(m_id)
    bp = 0
    ap = 0
    bb = 0
    ab = 0
    for inn in df_sim[df_sim.match_id==m_id].innings.unique():
        bp = 0
        ap = 0
        bb = 0
        ab = 0
        for index,row in df_sim[df_sim.match_id==m_id][df_sim.innings==inn].iterrows():
            ap = bp + row['total_runs']
            ab = bb + row['is_faced_by_batter']
            #print(ap,bp)
            df_sim.at[index, 'before_ball_partnership'] = bp
            df_sim.at[index, 'after_ball_partnership'] = ap
            df_sim.at[index, 'before_balls'] = bb
            df_sim.at[index, 'after_balls'] = ab
            bp = ap
            bb = ab
            if row['isWicket']==1:
                bp = 0
                ap = 0
                bb = 0
                ab = 0
        

p_sub = df_sim[['wickets_down','after_ball_partnership']]\
                .sort_values(['wickets_down','after_ball_partnership'], ascending=[True,False])

p_df = p_sub.groupby('wickets_down')['after_ball_partnership'].head(5).reset_index()

#p_df

/var/folders/68/nlclwstn1zd28pll4j60q7zw0000gn/T/ipykernel_81881/1875421427.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  for index,row in df_sim[df_sim.match_id==m_id][df_sim.innings==inn].iterrows():


In [8]:
df_sim['over_1'] = df_sim['legal_balls_bowled']//6
df_sim['over_2'] = df_sim['legal_balls_bowled']%6
df_sim['over_1'] = df_sim['over_1'].astype(str)
df_sim['over_2'] = df_sim['over_2'].astype(str)

df_sim['over'] = df_sim['over_1'] + "." +  df_sim['over_2']
df_sim['over'] = df_sim['over'].astype(float)

df_sim.drop(['over_1','over_2'],axis=1,inplace=True)
df_sim['over_'] = np.ceil(df_sim['over'])
df_sim['over_'] = df_sim['over_'].astype(int)

df_sim['phase'] = np.where(df_sim['over']<=6, "PP", np.where(df_sim['over']<=15, "Middle", "Death"))

In [9]:
df_sim = df_sim.merge(matches_list[['match_id','Spin']], on='match_id', how='left')
df_sim['pitch'] = df_sim['Spin'].apply(lambda x: 'pace' if x<=40 
                                       else 'spin' if x>=60
                                      else 'neutralish')

df_sim.sample(5)

In [10]:
team = 'RCB'

df_sim = df_sim[(df_sim.batting_team==team)|(df_sim.bowling_team==team)][df_sim.match_id.str.startswith('S')]

/var/folders/68/nlclwstn1zd28pll4j60q7zw0000gn/T/ipykernel_81881/3337514284.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_sim = df_sim[(df_sim.batting_team==team)|(df_sim.bowling_team==team)][df_sim.match_id.str.startswith('S')]


## spin-pace stats

In [11]:
df_sim = df_sim.merge(play_style_df, on='bowler',
                       how='left')


spin_bat = df_sim[df_sim['bowl_style_parent']=='Spin'].groupby('striker').agg(
    runs = ('runs_off_bat','sum'),
    balls = ('is_faced_by_batter','sum'),
    outs = ('isBowlerWicket','sum'),
    fours = ('isFour','sum'),
    sixes = ('isSix','sum'),
    dots = ('isDotforBatter','sum')
).reset_index()

spin_bat['SR'] = 100*spin_bat['runs']/spin_bat['balls']
spin_bat['Bat_avg'] = spin_bat['runs']/spin_bat['outs']
spin_bat['bpb'] = spin_bat['balls']/(spin_bat['fours']+spin_bat['sixes'])
spin_bat['dot_%'] = 100*spin_bat['dots']/spin_bat['balls']

pace_bat = df_sim[df_sim['bowl_style_parent']=='Pace'].groupby('striker').agg(
    runs = ('runs_off_bat','sum'),
    balls = ('is_faced_by_batter','sum'),
    outs = ('isBowlerWicket','sum'),
    fours = ('isFour','sum'),
    sixes = ('isSix','sum'),
    dots = ('isDotforBatter','sum')
).reset_index()

pace_bat['SR'] = 100*pace_bat['runs']/pace_bat['balls']
pace_bat['Bat_avg'] = pace_bat['runs']/pace_bat['outs']
pace_bat['bpb'] = pace_bat['balls']/(pace_bat['fours']+pace_bat['sixes'])
pace_bat['dot_%'] = 100*pace_bat['dots']/pace_bat['balls']
pace_bat['bowling_type'] = 'pace'
pace_bat['pace_index_int'] = pace_bat['SR']+np.where(pace_bat['Bat_avg']<1000, pace_bat['Bat_avg'], \
                                pace_bat['runs'])- 10*np.where(pace_bat['bpb']<1000, pace_bat['bpb'], \
                                        pace_bat['balls'])-pace_bat['dot_%']

pace_bat['pace_idx'] = np.where(
    ((pace_bat['balls'] < 300) ), #& (pace_bat['balls'] >= 50) 
    pace_bat['pace_index_int'] * ((pace_bat['balls'] / 300) ** 0.5),\
    np.where(pace_bat['balls'] >= 300, 
    pace_bat['pace_index_int'], \
            np.nan)
)


##
spin_bat['spin_index_int'] = spin_bat['SR']+np.where(spin_bat['Bat_avg']<1000, spin_bat['Bat_avg'], \
                                spin_bat['runs'])- 10*np.where(spin_bat['bpb']<1000, spin_bat['bpb'], \
                                        spin_bat['balls'])-spin_bat['dot_%']
spin_bat['bowling_type'] = 'spin'
spin_bat['spin_idx'] = np.where(
    ((spin_bat['balls'] < 300) ),  #& (spin_bat['balls'] >= 50) 
    spin_bat['spin_index_int'] * ((spin_bat['balls'] / 300) ** 0.5),\
    np.where(spin_bat['balls'] >= 300, 
    spin_bat['spin_index_int'], \
            np.nan)
)

##

k1 = 0.02
k2 = 0.1
c1 = 70
c2 = 97

def sigmoid(x):
    if x<=70:
        value = 0.5 + 1 / (1 + np.exp(-k1 * (x-c1)))
    else:
        value = 1 + 1/ (1 + np.exp(-k2 * (x-c2 )))
        
    return value

pace_bat['pace_index'] = pace_bat['pace_idx'].apply(sigmoid)
spin_bat['spin_index'] = spin_bat['spin_idx'].apply(sigmoid)

##
pace_bat.drop(['pace_index_int','pace_idx'], axis=1, inplace=True)
spin_bat.drop(['spin_index_int','spin_idx'], axis=1, inplace=True)



pl_st_df = pd.concat([pace_bat,spin_bat], axis=0)
pl_st_df = pl_st_df.round(2)

In [12]:
str_ = 'Gaikwad|Salt|Sudharsan|RK Singh|McGurk|Green|JM Sharma'
pl_st_df[pl_st_df['striker'].str.contains(str_)]\
        .sort_values('bowling_type',ascending=False)

,striker,runs,balls,outs,fours,sixes,dots,SR,Bat_avg,bpb,dot_%,bowling_type,pace_index,spin_index
5,B Sai Sudharsan,196,139,4,10,10,40,141.01,49.00,6.95,28.78,spin,NaN,0.96
8,C Green,72,56,1,4,5,22,128.57,72.00,6.22,39.29,spin,NaN,0.87
18,J Fraser-McGurk,87,56,1,6,4,15,155.36,87.00,5.60,26.79,spin,NaN,0.99
19,JM Sharma,46,32,3,5,2,13,143.75,15.33,4.57,40.62,spin,NaN,0.78
34,PD Salt,200,123,6,20,10,45,162.60,33.33,4.10,36.59,spin,NaN,1.11
38,RD Gaikwad,334,201,4,30,12,41,166.17,83.50,4.79,20.40,spin,NaN,1.99
42,RK Singh,115,88,4,7,4,27,130.68,28.75,8.00,30.68,spin,NaN,0.79
10,B Sai Sudharsan,148,107,5,11,9,47,138.32,29.60,5.35,43.93,pace,0.86,NaN
14,C Green,122,68,2,16,2,14,179.41,61.00,3.78,20.59,pace,1.26,NaN
27,J Fraser-McGurk,108,64,4,9,5,21,168.75,27.00,4.57,32.81,pace,0.92,NaN


## pitch-wise bowling stats

In [13]:
bowl_list = 'Pathirana|Brar|Nortje|Mujeeb|Mohsin|Chahal|Siraj|Green|RD Chahar|Malik|Nalkande|MM Ali'

pitch_ball = df_sim[df_sim.bowler.str.contains(bowl_list)].groupby(['bowler','pitch'])\
                        [['runs_conceeded','islegal','isBowlerWicket']].sum().reset_index()

pitch_ball['eco'] = 6*pitch_ball['runs_conceeded']/pitch_ball['islegal']
pitch_ball['SR'] = pitch_ball['islegal']/pitch_ball['isBowlerWicket']

pitch_ball = pitch_ball.rename(columns = {'islegal':'balls','runs_conceeded':'runs_conceded',
                                          'isBowlerWicket':'wkts'})

In [16]:
pitch_ball[pitch_ball.pitch=='spin'].round(2)

,bowler,pitch,runs_conceded,balls,wkts,eco,SR
2,A Nortje,spin,29,15,0,11.60,inf
4,C Green,spin,53,24,0,13.25,inf
8,Harpreet Brar,spin,195,120,6,9.75,20.00
11,M Pathirana,spin,156,95,4,9.85,23.75
12,MM Ali,spin,78,48,0,9.75,inf
18,Mujeeb Ur Rahman,spin,144,96,7,9.00,13.71
19,RD Chahar,spin,113,72,1,9.42,72.00
23,YS Chahal,spin,166,120,3,8.30,40.00


## inning-wise batter stats

In [17]:
inn_bat = df_sim.groupby(['striker','innings'])[['runs_off_bat','is_faced_by_batter']].sum().reset_index()
out_df = df_sim[df_sim.isWicket==1].groupby(['player_dismissed','innings'])['isWicket'].sum().reset_index()

out_df.columns= ['striker','innings','outs']
inn_bat = inn_bat.merge(out_df, on=['striker','innings'], how='outer')

inn_bat = inn_bat.rename(columns = {'runs_off_bat':'runs',
                                        'is_faced_by_batter':'balls'})

inn_bat['avg'] = inn_bat['runs']/inn_bat['outs']
inn_bat['SR'] = 100*inn_bat['runs']/inn_bat['balls']

inn_bat['runs'] = inn_bat['runs'].fillna(0).astype(int)
inn_bat['balls'] = inn_bat['balls'].fillna(0).astype(int)
inn_bat['outs'] = inn_bat['outs'].fillna(0).astype(int)
inn_bat = inn_bat.round(2)


In [19]:
#pat_ = 'KL|Livi|Markram|Lomror|VR Iyer|Duckett|Simran'
pat_ = 'Gaikwad|Salt|Sudharsan|RK Singh|McGurk|Green|JM Sharma'

inn_bat[(inn_bat['striker'].str.contains(pat_))]\
                    .sort_values('innings',ascending=False)

,striker,innings,runs,balls,outs,avg,SR
16,B Sai Sudharsan,2,108,87,6,18.00,124.14
21,C Green,2,108,76,3,36.00,142.11
39,J Fraser-McGurk,2,0,1,1,0.00,0.00
42,JM Sharma,2,48,19,3,16.00,252.63
69,PD Salt,2,305,194,6,50.83,157.22
78,RD Gaikwad,2,246,161,5,49.20,152.80
83,RK Singh,2,142,97,4,35.50,146.39
15,B Sai Sudharsan,1,236,159,5,47.20,148.43
20,C Green,1,86,48,1,86.00,179.17
38,J Fraser-McGurk,1,195,119,4,48.75,163.87
